In [1]:
#!pip install kagglehub

GET AND CHECK DATA

In [2]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("darshue/extracted-images")

# print("Path to dataset files:", path)

If you're training the model using colab use the bottom  command to install the datasets. You'll need your kaggle api key which you can generate in your account settings at kaggle. Download the api key and paste the key when prompted to download the dataset

In [3]:
#!pip install opendatasets
# import opendatasets as od
# od.download("https://www.kaggle.com/datasets/darshue/extracted-images")

The code below checks if youre dataset has been installed at the correct path

In [4]:
import os

is_colab = False
if is_colab:
    data_dir = './extracted-images/train/'
else:
    data_dir = "../data/train/"


print("Path exists:", os.path.exists(data_dir))  # Should print True
print("Contents:", os.listdir(data_dir))  # Should list the 5 subclasses

output_dir = '../output'
os.makedirs(output_dir, exist_ok=True)

Path exists: True
Contents: ['CC', 'EC', 'HGSC', 'LGSC', 'MC']


IMPORTS AND DEVICE

In [5]:
import torch
import os
import os.path
import numpy as np
import sys
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import tqdm
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.datasets as datasets
import seaborn as sns
from sklearn.metrics import confusion_matrix

np.random.seed(111)
torch.manual_seed(111)
torch.cuda.manual_seed_all(111)

In [6]:
if torch.cuda.is_available():
    print("CUDA available")
    device = torch.device("cuda")
else:
    print("CUDA not available")
    device = torch.device("cpu")
print("Device: ", device)

CUDA available
Device:  cuda


In [ ]:
train_test_split = 0.8
batch_size = 32
num_epochs = 50
learning_rate = 0.001
num_classes = 5
if is_colab:
    num_workers = 4
else:
    num_workers = 1


In [8]:
mean, std = [0.7276], [0.1001]

In [9]:
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((496, 496)),
    transforms.RandomCrop(480),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((496, 496)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [10]:
train_dataset = datasets.ImageFolder(root=os.path.join(data_dir), transform=train_transform)
test_dataset = datasets.ImageFolder(root=os.path.join(data_dir), transform=test_transform)

train_size = int(train_test_split * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print('Number of training samples:', len(train_loader.dataset))
print('Number of testing samples:', len(test_loader.dataset))

classes = [ 'CC', 'EC', 'HGSC', 'LGSC', 'MC' ]

Number of training samples: 20402
Number of testing samples: 5101


In [11]:
# Used to get mean and std of dataset, does not need to be run again
if False:
    mean_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((496, 496)),
    transforms.ToTensor()
    ])
    dataset = datasets.ImageFolder(root=os.path.join(data_dir), transform=mean_transform)

    loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=num_workers)

    mean = 0.0
    std = 0.0
    nb_samples = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        # Flatten images in each batch: (batch_size, channels, height * width)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples

    print('Mean:', mean)
    print('Std:', std)

In [12]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model = model.to(device)

c:\Users\Jordan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Jordan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.00025)

# Learning rate scheduler implementation
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

In [14]:
# Store values for plot
epoch_losses = []
val_accuracies = []
test_accuracies = []


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    #Training Loop
    loop = tqdm.tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    scheduler.step() #LR implementation after every epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_losses.append(epoch_loss) # store data to plot

    #Validation Accuray
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        val_loop = tqdm.tqdm(test_loader, desc="Validation")
        for images, labels in val_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = outputs.argmax(dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    val_accuracies.append(val_accuracy)

    # Test Accuray
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = outputs.argmax(dim=1)
            correct_test += (predictions == labels).sum().item()
            total_test += labels.size(0)
    test_accuracy = correct_test / total_test
    test_accuracies.append(test_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

model_path = os.path.join(output_dir, 'resnet18_model.pth')
torch.save(model.state_dict(), model_path)
print("Model saved to:", model_path)

Epoch [1/10]:  29%|██▉       | 188/638 [00:55<02:12,  3.40it/s, loss=0.886]


KeyboardInterrupt: 

In [ ]:
plt.figure

#loss plot
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), epoch_losses, marker='o', linestyle='-', color='b', label="Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.legend()

# Accuracy Plot
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), val_accuracies, marker='o', linestyle='-', color='g', label="Validation Accuracy")
plt.plot(range(1, num_epochs + 1), test_accuracies, marker='s', linestyle='-', color='r', label="Test Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Validation & Test Accuracy Over Epochs")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model.eval()

all_preds = []
all_labels = []


with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)

classes = ['CC', 'EC', 'HGSC', 'LGSC', 'MC']

# Plot
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()